In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_7033dff430354afa90a1a39a6fa0689a.csv"
df=pd.read_csv(url,header=None,na_values=["*"])
df.head()

,0,1,2,3,4,5,6
0,2,NaN,NaN,NaN,NaN,NaN,2
1,1,2.0,NaN,NaN,NaN,NaN,1
2,1,1.0,2.0,NaN,NaN,NaN,1
3,1,1.0,1.0,NaN,NaN,NaN,1
4,1,1.0,3.0,2.0,2.0,NaN,1


In [2]:
df.shape

(15, 7)

In [3]:
df.dtypes

0      int64
1    float64
2    float64
3    float64
4    float64
5    float64
6      int64
dtype: object

In [4]:
df.mean()

0    1.600000
1    1.076923
2    3.000000
3    1.142857
4    1.571429
5    2.200000
6    1.066667
dtype: float64

In [5]:
df=df.fillna(0)
df.head()

,0,1,2,3,4,5,6
0,2,0.0,0.0,0.0,0.0,0.0,2
1,1,2.0,0.0,0.0,0.0,0.0,1
2,1,1.0,2.0,0.0,0.0,0.0,1
3,1,1.0,1.0,0.0,0.0,0.0,1
4,1,1.0,3.0,2.0,2.0,0.0,1


In [6]:
y=pd.DataFrame(df.iloc[:,0]).rename(columns={0:"Class"})
y.head()

,Class
0,2
1,1
2,1
3,1
4,1


In [7]:
y.groupby("Class")["Class"].count()

Class
1    6
2    9
Name: Class, dtype: int64

In [8]:
import numpy as np
X=pd.DataFrame(np.asarray(df.iloc[:,1:7]))
X.head()

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,2.0
1,2.0,0.0,0.0,0.0,0.0,1.0
2,1.0,2.0,0.0,0.0,0.0,1.0
3,1.0,1.0,0.0,0.0,0.0,1.0
4,1.0,3.0,2.0,2.0,0.0,1.0


In [9]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
1,4.897674
3,0.681283
5,0.650000
4,0.471141
0,0.200000
2,0.025243


In [10]:
X_scores=scores[scores["scores"]>0.2]
len(X_scores)

4

In [11]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,1,3,5,4
0,0.0,0.0,2.0,0.0
1,0.0,0.0,1.0,0.0
2,2.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0
4,3.0,2.0,1.0,0.0


In [12]:
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [13]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[90,95,100,105,110,115], 'criterion':("gini","entropy"),
             'min_samples_split':[2,3,4,5],'min_samples_leaf':[1,2,3,4],'max_features':("sqrt","log2","auto")}

In [14]:
Forest=RandomForestClassifier(random_state=0)
Forest_grid = GridSearchCV(estimator =Forest, param_grid = parameters,cv = 3,scoring="roc_auc", verbose=2, n_jobs = -1)

In [15]:
Forest_grid.fit(X,np.asarray(y).reshape(y.shape[0],))

Fitting 3 folds for each of 576 candidates, totalling 1728 fits
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=90 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=90, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=90 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=90, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=90 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=90, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=95 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=95, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=95 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=95, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=95 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=95, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min

[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=95, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=95 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=95, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=100 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=100, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=100 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=100, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=100 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=100, total=   0.2s
[CV] criterion=gini, max_features=sqr

[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=105 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=105, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=105 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=105, total=   0.2s
[CV] criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=105 
[CV]  criterion=gini, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=105, total=   0.2s
[CV] criterion=gini, max_features=

...

[CV]  criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=95, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=95 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=95, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=100 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=100, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=100 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=100, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=100 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=100, total=   0.2s
[CV] crite

[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=100, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=100 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=100, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=100 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=100, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=105 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=105, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=105 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=3, n_estimators=105, total=   0.2s
[CV] cr

[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=105, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=105 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=105, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=105 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=105, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=110 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=110, total=   0.2s
[CV] criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=110 
[CV]  criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=110, total=   0.2s
[CV] cr

[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed:  5.0min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_features': ('sqrt', 'log2', 'auto'),
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [90, 95, 100, 105, 110, 115]},
             scoring='roc_auc', verbose=2)

In [16]:
Forest_grid.best_params_

{'criterion': 'gini',
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 95}

In [17]:
best_grid = Forest_grid.best_estimator_

In [18]:
scores=cross_val_score(best_grid,X,np.asarray(y).reshape(y.shape[0],),cv=2)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.5982142857142857 +/- 0.0267857142857143


In [19]:
y_pred = cross_val_predict(best_grid, X,np.asarray(y).reshape(y.shape[0],), cv=2)
AUC = auc(y, y_pred)
print("AUC: ",AUC)

AUC:  0.5


In [27]:
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[0 6]
 [0 9]]


#### The results from the grid search are worse than the results produced by the values I intuitively put in.

In [20]:
Forest=RandomForestClassifier(random_state=0)
scores=cross_val_score(Forest,Xp,np.asarray(y).reshape(y.shape[0],),cv=2)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7321428571428572 +/- 0.01785714285714285


In [21]:
scores

array([0.75 , 0.714])

In [38]:
y_pred = cross_val_predict(Forest, Xp,np.asarray(y).reshape(y.shape[0],), cv=2)
AUC = auc(y, y_pred,average="macro")
print("AUC: ",AUC)

AUC:  0.7222222222222223


In [39]:
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[4 2]
 [2 7]]


In [24]:
from sklearn.ensemble import GradientBoostingClassifier
Gradient=GradientBoostingClassifier(random_state=0)
scores=cross_val_score(Gradient,Xp,np.asarray(y).reshape(y.shape[0],),cv=2)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7321428571428572 +/- 0.01785714285714285


In [40]:
y_pred = cross_val_predict(Gradient, Xp,np.asarray(y).reshape(y.shape[0],), cv=2)
AUC = auc(y, y_pred,average="macro")
print("AUC: ",AUC)

AUC:  0.7222222222222223


In [41]:
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[4 2]
 [2 7]]


In [27]:
from imblearn.ensemble import BalancedRandomForestClassifier
BForest=BalancedRandomForestClassifier(n_jobs=-1,warm_start=False,sampling_strategy="not majority",
                                       random_state=0)
scores=cross_val_score(BForest,Xp,np.asarray(y).reshape(y.shape[0],),cv=2)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7321428571428572 +/- 0.01785714285714285


In [42]:
y_pred = cross_val_predict(BForest, Xp,np.asarray(y).reshape(y.shape[0],), cv=2)
AUC = auc(y, y_pred,average="macro")
print("AUC: ",AUC)

AUC:  0.7222222222222223


In [29]:
from imblearn.ensemble import BalancedBaggingClassifier
BBagging=BalancedBaggingClassifier(base_estimator=RandomForestClassifier(),sampling_strategy="all",random_state=0)
scores=cross_val_score(BBagging,Xp,np.asarray(y).reshape(y.shape[0],),cv=2)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8035714285714286 +/- 0.05357142857142855


In [30]:
scores

array([0.75 , 0.857])

In [43]:
y_pred = cross_val_predict(BBagging, Xp,np.asarray(y).reshape(y.shape[0],), cv=2)
AUC = auc(y, y_pred,average="macro")
print("AUC: ",AUC)

AUC:  0.8055555555555555


In [35]:
BBagging2=BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),sampling_strategy="auto",random_state=0)
scores=cross_val_score(BBagging2,Xp,np.asarray(y).reshape(y.shape[0],),cv=2)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8035714285714286 +/- 0.05357142857142855


In [44]:
y_pred = cross_val_predict(BBagging2, Xp,np.asarray(y).reshape(y.shape[0],), cv=2)
AUC = auc(y, y_pred,average="macro")
print("AUC: ",AUC)

AUC:  0.8055555555555555


In [45]:
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[5 1]
 [2 7]]


#### Summary

|Model|Accuracy|AUC|
|-----|--------|---|
|RandomForestClassifier, 2-fold cross validation|0.7321 +/- 0.0179|0.7222|
|GradientBoostingClassifier, 2-fold cross validation|0.7321 +/- 0.0179|0.7222|
|BalancedRandomForestClassifier, 2-fold cross validation|0.7321 +/- 0.0179|0.7222|
|BalancedBaggingClassifier with RandomForestClassifier as base estiamtor|0.8035 +/- 0.0535|0.8055|
|BalancedBaggingClassifier with GradidentBoostingClassifier as base estimator|0.8035 +/- 0.0535|0.8055|